<a href="https://colab.research.google.com/github/arcweld/DSI_Capstone_3/blob/main/notebooks/8_smw_nlcd_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geojson
!pip install geemap
!pip install geopandas

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import json
import geojson
import requests

import geopandas as gpd
import numpy as np
import pandas as pd

import ee
import geemap
import geemap.eefolium as emap
import folium

import os

In [3]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=eHFASWkA8tRHOjuyaj13iBZ-XekQCl9w4Bc6mfFTFCM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhe-E4YkpI69071DSJxMlvaSR7EXBFtpHu9WHDDOa4tr6_v41i4Rgk

Successfully saved authorization token.


In [4]:
from google.colab import drive
drive.mount('/data', force_remount=True)

Mounted at /data


In [20]:
Map = geemap.Map()

In [21]:
point = ee.Geometry.Point([-91.8775, 33.9717])

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(point) \
        .filterDate('2019-01-01', '2019-12-31') \
        .sort('CLOUD_COVER') \
        .first() \
        .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 9)
Map.addLayer(image, vis_params, 'Landsat8')


In [22]:
props = geemap.image_props(image)
props.getInfo()

{'CLOUD_COVER': 0.01,
 'CLOUD_COVER_LAND': 0.01,
 'EARTH_SUN_DISTANCE': 1.004895,
 'ESPA_VERSION': '2_23_0_1b',
 'GEOMETRIC_RMSE_MODEL': 6.949,
 'GEOMETRIC_RMSE_MODEL_X': 4.983,
 'GEOMETRIC_RMSE_MODEL_Y': 4.843,
 'IMAGE_DATE': '2019-04-21',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_024036_20190421_20190507_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1557255705000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2019-04-21T16:42:41.0529550Z',
 'SOLAR_AZIMUTH_ANGLE': 134.73703,
 'SOLAR_ZENITH_ANGLE': 29.686024,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 24,
 'WRS_ROW': 36,
 'system:asset_size': '624.750103 MB',
 'system:band_names': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_024036_20190421',
 'system:index': 'LC08_024036_20190421',
 'system:time_end': '2019-04-21 16:42:41',
 'system:time_start': '2019-04-21 16:42:41',
 'system:version': 15633957374

In [23]:
props.get('IMAGE_DATE').getInfo()

'2019-04-21'

In [24]:
props.get('CLOUD_COVER').getInfo()

0.01

In [25]:
training = image.sample(**{
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True
})

Map.addLayer(training, {}, 'training', False)

In [26]:
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)
result = image.cluster(clusterer)
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

In [27]:
legend_keys = ['One', 'Two', 'Three', 'Four', 'Five']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']

result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(result, {'min': 1, 'max': 5, 'palettte': legend_colors}, 'Labelled Clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
Map.addLayerControl()
Map

In [31]:
Map = geemap.Map()

In [33]:
point = ee.Geometry.Point([-79.091252, 40.347754])

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                            .filterBounds(point) \
                            .filterDate('2016-01-01', '2016-12-31') \
                            .sort('CLOUD_COVER') \
                            .first() \
                            .select('B[1-7]')
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 9)
Map.addLayer(image, vis_params, 'Landsat8')

In [34]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2016-04-20'

In [36]:
image.get('CLOUD_COVER').getInfo()

0.01

In [37]:
nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())
Map.addLayer(nlcd, {}, 'NLCD')

In [39]:
points = nlcd.sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True
})

Map.addLayer(points, {}, 'training', False)

In [41]:
points.size().getInfo()

5000

In [42]:
points.first().getInfo()

{'geometry': {'coordinates': [-77.73612934519153, 40.649091305295954],
  'type': 'Point'},
 'id': '0',
 'properties': {'landcover': 41},
 'type': 'Feature'}

In [43]:
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

label = 'landcover'

training = image.select(bands).sampleRegions(**{
    'collection': points,
    'properties': [label],
    'scale': 30
})

trained = ee.Classifier.smileCart().train(training, label, bands)

In [44]:
training.first().getInfo()

{'geometry': None,
 'id': '0_0',
 'properties': {'B1': 269,
  'B2': 314,
  'B3': 448,
  'B4': 582,
  'B5': 1485,
  'B6': 2117,
  'B7': 1406,
  'landcover': 41},
 'type': 'Feature'}

In [45]:
result = image.select(bands).classify(trained)

Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map

In [47]:
class_values = nlcd.get('landcover_class_values').getInfo()
class_values

[11,
 12,
 21,
 22,
 23,
 24,
 31,
 41,
 42,
 43,
 51,
 52,
 71,
 72,
 73,
 74,
 81,
 82,
 90,
 95]

In [48]:
class_palette = nlcd.get('landcover_class_palette').getInfo()
class_palette

['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94',
 'dcd93d',
 'ab7028',
 'bad9eb',
 '70a3ba']

In [49]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [50]:
Map.addLayer(landcover, {}, 'Land cover')
Map

In [51]:
Map.add_legend(builtin_legend='NLCD')
Map

In [52]:
out_dir = os.path.join('../data/MyDrive/Galvanize/DSI_Capstone_3/reports/figures')
out_file = os.path.join(out_dir, 'NLCD.tif')

In [53]:
geemap.ee_export_image(landcover, filename=out_file, scale=900)

Generating URL ...
Please wait ...
Data downloaded to /data/MyDrive/Galvanize/DSI_Capstone_3/reports/figures/NLCD.tif


accuracy assessment

In [66]:
Map = geemap.Map()

In [67]:
NLCD2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
Map.addLayer(NLCD2016, {}, 'NLCD 2016')

In [68]:
NLCD_metadata = ee.FeatureCollection("users/giswqs/landcover/NLCD2016_metadata")
Map.addLayer(NLCD_metadata, {}, 'NLCD Metadata')

In [69]:
point = ee.Geometry.Point([-88.3070, 41.7471])     # Chicago, IL

In [70]:
metadata = NLCD_metadata.filterBounds(point).first()
region = metadata.geometry()

In [71]:
metadata.get('2016on_bas').getInfo()

'LC08_2016256'

In [72]:
doy = metadata.get('2016on_bas').getInfo().replace('LC08_', '')
doy

'2016256'

In [73]:
ee.Date.parse('YYYYDDD', doy).format('YYYY-MM-dd').getInfo()

'2016-09-12'

In [74]:
start_date = ee.Date.parse('YYYYDDD', doy)
end_date = start_date.advance(1, 'day')

In [75]:
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate(start_date, end_date) \
    .first() \
    .select('B[1-7]') \
    .clip(region)

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

In [76]:
nlcd_raw = NLCD2016.clip(region)
Map.addLayer(nlcd_raw, {}, 'NLCD')

In [77]:
raw_class_values = nlcd_raw.get('landcover_class_values').getInfo()
print(raw_class_values)

[11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95]


In [78]:
n_classes = len(raw_class_values)
new_class_values = list(range(0, n_classes))
new_class_values

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [79]:
class_palette = nlcd_raw.get('landcover_class_palette').getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [80]:
nlcd = nlcd_raw.remap(raw_class_values, new_class_values).select(['remapped'], ['landcover'])
nlcd = nlcd.set('landcover_class_values', new_class_values)
nlcd = nlcd.set('landcover_class_palette', class_palette)

In [81]:
Map.addLayer(nlcd, {}, 'NLCD')
Map

In [82]:
# Make the training dataset.
points = nlcd.sample(**{
    'region': region,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)

In [83]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
sample = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Adds a column of deterministic pseudorandom numbers. 
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [84]:
training.first().getInfo()

{'geometry': None,
 'id': '2_0',
 'properties': {'B1': 72,
  'B2': 215,
  'B3': 579,
  'B4': 543,
  'B5': 2558,
  'B6': 1782,
  'B7': 1103,
  'landcover': 3,
  'random': 0.18158720869622524},
 'type': 'Feature'}

In [85]:
validation.first().getInfo()

{'geometry': None,
 'id': '0_0',
 'properties': {'B1': 174,
  'B2': 223,
  'B3': 614,
  'B4': 368,
  'B5': 4221,
  'B6': 1737,
  'B7': 766,
  'landcover': 17,
  'random': 0.889822614157854},
 'type': 'Feature'}

In [86]:
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [87]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

In [88]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [89]:
Map.addLayer(landcover, {}, 'Land cover')
Map

In [90]:
Map.add_legend(builtin_legend='NLCD')
Map

In [91]:
train_accuracy = classifier.confusionMatrix()

In [92]:
train_accuracy.getInfo()

[[272, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 183, 6, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 0, 0],
 [0, 0, 1, 421, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0],
 [1, 0, 1, 15, 171, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 3, 8, 96, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 2, 2, 0, 0, 0, 212, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 2, 0, 13, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 32, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [93]:
train_accuracy.accuracy().getInfo()

0.9562445667922341

In [94]:
validated = validation.classify(classifier)
validated.first().getInfo()

{'geometry': None,
 'id': '0_0',
 'properties': {'B1': 174,
  'B2': 223,
  'B3': 614,
  'B4': 368,
  'B5': 4221,
  'B6': 1737,
  'B7': 766,
  'classification': 17,
  'landcover': 17,
  'random': 0.889822614157854},
 'type': 'Feature'}

In [95]:
test_accuracy = validated.errorMatrix('landcover', 'classification')
test_accuracy.getInfo()

[[145, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 13, 18, 2, 0, 0, 15, 0, 0, 0, 0, 1, 0, 0, 0, 2, 54, 1, 0],
 [2, 0, 8, 113, 20, 2, 1, 7, 0, 0, 0, 0, 1, 0, 0, 0, 3, 33, 3, 0],
 [1, 0, 1, 27, 34, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 1, 0],
 [0, 0, 1, 2, 18, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 12, 6, 1, 0, 0, 54, 0, 0, 0, 0, 0, 0, 0, 0, 1, 14, 5, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 10, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [96]:
test_accuracy.accuracy().getInfo()

0.7023886378308586

In [97]:
landcover = landcover.remap(new_class_values, raw_class_values).select(['remapped'], ['classification'])

In [98]:
landcover = landcover.set('classification_class_values', raw_class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [99]:
Map.addLayer(landcover, {}, 'Final land cover')
Map